In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [2]:
url = "https://www.britannica.com/topic/Wikipedia"

In [3]:
d = {"key": 'value'}
print(d)

{'key': 'value'}


In [4]:
d["new key"] = 'new value'
d

{'key': 'value', 'new key': 'new value'}

In [9]:
npo_jobs = {}
job_no = 0

while True:

    response = requests.get(url)
    data = response.text
    soup = BeautifulSoup(data, "html.parser")
    jobs = soup.find_all("p", {"class": "result-info"})


    for job in jobs:
        title = job.find('a', {'class': 'result-title'}).text
        location_tag = job.find('span', {'class': 'result-hood'})
        location = location_tag.text[2: -1] if location_tag else "N/A"
        date = job.find('time', {'class': 'result-date'}).text
        link = job.find('a', {'class': "result-title"}).get("href")
        job_response = requests.get(link)
        job_data = job_response.text
        job_soup = BeautifulSoup(job_data, 'html.parser')
        job_description = job_soup.fing('section', {'id': "postingbody"}).text
        job_attributes_tag = job_soup.find("p", {"class": "attrgroup"})
        job_attributes = job_attributes_tag.text if job_attributes_tag else "N/A"

        job_no += 1
        npo_jobs[job_no] = [title, location, date, link, job_attributes, job_description]


        print("Job Title:", title, "\nLocation:", location, "\nDate:", date, "\nLink:", link, "\n", job_attributes, "\nJob Dscription", job_description)

    
    url_tag = soup.find("a", {"title": "next page"})
    if url_tag is not None and url_tag.get("href"):
        url = "https://www.britannica.com/topic/Wikipedia" + url_tag.get("href")
        print(url)
    else:
        break

print("Total Jobs:", job_no)
npo_jobs_df = pd.DataFrame.from_dict(npo_jobs, orient= "index", columns= ['Job Title', 'Location', 'Date', 'Link', 'Job attributes', 'Job Description'])

Total Jobs: 0


In [10]:
npo_jobs_df.head()

,Job Title,Location,Date,Link,Job attributes,Job Description


In [11]:
npo_jobs_df.to_csv('npo_jobs.csv')